# Part 0 Scarper


In [ ]:
import warnings
import GoogleNews
import pandas as pd
from datetime import datetime, timedelta
import time
import random
import os
import glob
import sys  # 引入 sys 以便強制中止程式

# from google.colab import drive
# drive.mount('/content/drive')
# drive_folder = '/content/drive/MyDrive/fiisual/news_data2'

current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)
file_path = os.path.join(project_root, 'data', 'archive', 'all_data.csv')

drive_folder = os.path.join(base_dir, 'fiisual', 'news_data2')
warnings.filterwarnings('ignore')

# ================= 📁 存檔與進度管理 =================


if not os.path.exists(drive_folder):
    os.makedirs(drive_folder)

def get_completed_dates(folder_path):
    """掃描資料夾，找出已經爬完的日期 (格式 YYYYMMDD)"""
    files = glob.glob(os.path.join(folder_path, 'yahoo_news_*.csv'))
    completed = set()
    for f in files:
        filename = os.path.basename(f)
        try:
            date_part = filename.split('_')[-1].replace('.csv', '')
            completed.add(date_part)
        except:
            pass
    return completed

# ================= 爬蟲設定 =================
days_back = 30
target_results_per_day = 100
pages_to_scrape = target_results_per_day // 10

# 基礎關鍵字
base_query = ('site:finance.yahoo.com/news '
        '("Bloomberg" OR "Reuters" OR "CNBC" ) '
        '-site:sg.finance.yahoo.com -site:ca.finance.yahoo.com -site:nz.finance.yahoo.com -site:uk.finance.yahoo.com -site:au.finance.yahoo.com '
        '-"The Motley Fool" -Zacks -TipRanks -"GuruFocus." '
        '-"Insider Monkey" -"InvestorPlace" -"Benzinga" -"Seeking Alpha" '
        '-"Press Release" -GlobeNewswire -"Business Wire" -"PR Newswire" -"Accesswire"'
        )

# ==========================================

# 設定基準日
end_date = datetime(2025, 12, 14)
date_list = [end_date - timedelta(days=x) for x in range(days_back)]

# 🔍 讀取已完成的進度
completed_dates = get_completed_dates(drive_folder)
print(f"📂 目前已存檔 {len(completed_dates)} 天的資料，將自動跳過這些日期。")
print(f"🐢 啟動 GoogleNews (偵測 429 自動停止版)... 存檔至：{drive_folder}\n")

for target_date in date_list:
    file_date_str = target_date.strftime('%Y%m%d')
    date_str = target_date.strftime('%m/%d/%Y')

    # === 🛑 檢查點：如果這天跑過了，就跳過 ===
    if file_date_str in completed_dates:
        print(f"⏭️ [{date_str}] 檔案已存在，跳過。")
        continue

    # 計算日期區間
    day_before = target_date - timedelta(days=1)
    day_after = target_date + timedelta(days=1)
    str_after = day_before.strftime('%Y-%m-%d')
    str_before = day_after.strftime('%Y-%m-%d')
    standard_date_str = target_date.strftime('%Y-%m-%d')



    print(f"📅 [{date_str}] 處理中... (目標: {target_results_per_day} 篇)")

    googlenews = GoogleNews(lang='en', region='US')
    final_query = f'{base_query} after:{str_after} before:{str_before}'

    # 嘗試初始化搜尋
    try:
        googlenews.search(final_query)

    except Exception as e:
            print(f"  初始化失敗 ")
            continue

    # === 爬取頁面 ===
    daily_results = []

    for page in range(1, pages_to_scrape):
        try:
            googlenews.get_page(page)

            # 隨機延遲 (稍微拉長一點點，降低被封鎖機率)
            sleep_time = random.uniform(1, 3)
            print(f"   -> Page {page} done. 休息 {int(sleep_time)}s...", end='\r')
            time.sleep(sleep_time)

        except Exception as e:
            error_msg = str(e)

            # === 🚨 關鍵修改：偵測 429 這裡最重要 ===
            if "429" in error_msg or "Too Many Requests" in error_msg:
                print("\n" + "="*40)
                print(f"🛑 偵測到 429 封鎖！(於第 {page} 頁)")
                print(f"🚨 請執行以下步驟：")
                print(f"   1. 選單【執行階段】->【中斷連線並刪除執行階段】")
                print(f"   2. 重新連線並執行程式，將從 [{date_str}] 繼續。")
                print("="*40 + "\n")
                sys.exit("IP Blocked - Stop Execution") # 強制停止
            else:
                print(f"\n   ⚠️ Page {page} 發生未知錯誤: {e} (嘗試繼續)")

    # === 存檔區塊 ===
    daily_data = googlenews.result()

    if daily_data:
        df = pd.DataFrame(daily_data)
        if 'title' in df.columns:
            # 1. 移除空標題與重複連結
            df = df[df['title'].astype(bool)]
            df = df.drop_duplicates(subset=['link'])

            df['date'] = standard_date_str
            # ====================================================

            # 存檔
            cols_to_keep = ['date', 'title', 'media', 'link'] # 把 date 放第一欄方便看
            df = df[cols_to_keep] # 只留這些欄位，其他雜亂的都丟掉

            filename = f'yahoo_news_{target_date.strftime("%Y%m%d")}.csv'
            full_path = os.path.join(drive_folder, filename)

            df.to_csv(full_path, index=False, encoding='utf-8-sig')
            print(f"\n✅ [{standard_date_str}] 存檔完成 ({len(df)} 筆)")
    else:
        print(f"\n⚠️ [{date_str}] 無資料。")

    print("☕ 換日休息 10 秒...\n")
    time.sleep(1)

print("🎉 全部任務完成！")

UndefVarError: UndefVarError: `pip` not defined in `Main`
Suggestion: check for spelling errors or missing imports.